# Use TextGrad with TogetherAI



# What is TextGrad?

TextGrad is a recent framework for the end-to-end optimization of language models prompts thorugh text feedback. What this basically means is that TextGrad will allow you to optimize language models' prompts and solutions automatically.

TogetherAI will give us the models to do this in an effective way!

In [9]:
import httpx
from textgrad.engine_experimental.litellm import LiteLLMEngine
import textgrad as tg
from dotenv import load_dotenv

load_dotenv()

True

How to use togetherAI with TextGrad

In [17]:
generation = LiteLLMEngine("together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo", cache=True).generate(content="hello, what's 3+4", system_prompt="you are an assistant")
generation

'Hello, the answer to 3 + 4 is 7. Is there anything else I can help you with?'

In [6]:
# We'll use below utilities to run a python function.
from IPython.core.interactiveshell import InteractiveShell

def run_function_in_interpreter(func_code):
    #raise Exception("This function will run the code returned by GPT-4o. Remove this if you'd like to run the code!")
    interpreter = InteractiveShell.instance()
    
    interpreter.run_cell(func_code, store_history=False, silent=True)
    
    func_name = func_code.split("def ")[1].split("(")[0].strip()
    func = interpreter.user_ns[func_name]
    
    return func


def test_longest_increasing_subsequence(fn):
    nums = [10, 22, 9, 33, 21, 50, 41, 60]
    assert fn(nums) == 5

    nums = [7, 2, 1, 3, 8, 4, 9, 6, 5]
    assert fn(nums) == 4

    nums = [5, 4, 3, 2, 1]
    assert fn(nums) == 1

    nums = [1, 2, 3, 4, 5]
    assert fn(nums) == 5

    nums = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
    assert fn(nums) == 4

    nums = [10, 9, 2, 5, 3, 7, 101, 18]
    assert fn(nums) == 4

    nums = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
    assert fn(nums) == 6

    nums = [7, 7, 7, 7, 7, 7, 7]
    assert fn(nums) == 1

    nums = [20, 25, 47, 35, 56, 68, 98, 101, 212, 301, 415, 500]
    assert fn(nums) == 11

    nums = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
    assert fn(nums) == 1

    print("All test cases passed!")

### Making Code Faster


In [7]:
import random
import time

problem_text = """Longest Increasing Subsequence (LIS)

Problem Statement:
Given a sequence of integers, find the length of the longest subsequence that is strictly increasing. A subsequence is a sequence that can be derived from another sequence by deleting some or no elements without changing the order of the remaining elements.

Input:
The input consists of a list of integers representing the sequence.

Output:
The output should be an integer representing the length of the longest increasing subsequence."""

initial_solution = """
def longest_increasing_subsequence(nums):
    n = len(nums)
    dp = [1] * n
    
    for i in range(1, n):
        for j in range(i):
            if nums[i] > nums[j]:
                dp[i] = max(dp[i], dp[j] + 1)
    
    max_length = max(dp)
    lis = []
    
    for i in range(n - 1, -1, -1):
        if dp[i] == max_length:
            lis.append(nums[i])
            max_length -= 1
    
    return len(lis[::-1])
"""

# Generate a random test case
def generate_random_test_case(size, min_value, max_value):
    return [random.randint(min_value, max_value) for _ in range(size)]

# Test the function with a random test case
size = 10000  # Adjust the size as needed
min_value = 1
max_value = 1000

nums = generate_random_test_case(size, min_value, max_value)


In [8]:
longest_increasing_subsequence = run_function_in_interpreter(initial_solution)

start_time = time.time()
lis = longest_increasing_subsequence(nums)
end_time = time.time()

print(f"Test Case Size: {size}")
print(f"Longest Increasing Subsequence Length: {lis}")
print(f"Runtime: {end_time - start_time:.5f} seconds")

# Test for all test cases
test_longest_increasing_subsequence(longest_increasing_subsequence)

Test Case Size: 10000
Longest Increasing Subsequence Length: 180
Runtime: 2.42051 seconds
All test cases passed!


In [26]:
llm_engine = LiteLLMEngine("together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo", cache=True)
tg.set_backward_engine(llm_engine, override=True)

# Code is the variable of interest we want to optimize -- so requires_grad=True
code = tg.Variable(value=initial_solution,
                requires_grad=True,
                role_description="code instance to optimize")

# We are not interested in optimizing the problem -- so requires_grad=False
problem = tg.Variable(problem_text, 
                    requires_grad=False, 
                    role_description="the coding problem")

# Let TGD know to update code!
optimizer = tg.TGD(parameters=[code])

In [27]:
# The system prompt that will guide the behavior of the loss function.
loss_system_prompt = "You are a smart language model that evaluates code snippets. You do not solve problems or propose new code snippets, only evaluate existing solutions critically and give very concise feedback."
loss_system_prompt = tg.Variable(loss_system_prompt, requires_grad=False, role_description="system prompt to the loss function")

# The instruction that will be the prefix
instruction = """Think about the problem and the code snippet. Does the code solve the problem? What is the runtime complexity?"""

# The format string and setting up the call
format_string = "{instruction}\nProblem: {{problem}}\nCurrent Code: {{code}}"
format_string = format_string.format(instruction=instruction)

fields = {"problem": None, "code": None}
formatted_llm_call = tg.autograd.FormattedLLMCall(engine=llm_engine,
                                                  format_string=format_string,
                                                  fields=fields,
                                                  system_prompt=loss_system_prompt)

# Finally, the loss function
def loss_fn(problem: tg.Variable, code: tg.Variable) -> tg.Variable:
    inputs = {"problem": problem, "code": code}
    
    return formatted_llm_call(inputs=inputs,
                              response_role_description=f"evaluation of the {code.get_role_description()}")



In [28]:
loss = loss_fn(problem, code)
print(loss.value)

The code solves the problem. 
Runtime complexity: O(n^2), where n is the length of the input sequence. 

Note: The code not only finds the length of the longest increasing subsequence but also attempts to reconstruct it. However, the reconstruction part is unnecessary as the problem only asks for the length. The line `return len(lis[::-1])` is redundant and can be simplified to `return max(dp)`.


In [29]:
loss.backward()
print(code.gradients)

{Variable(value=To improve the code instance, several adjustments can be made to enhance its efficiency and simplicity, directly impacting the objective function by reducing unnecessary computations and improving runtime complexity.

1. **Simplification of the Return Statement**: The current implementation not only finds the length of the longest increasing subsequence (LIS) but also attempts to reconstruct it, which is unnecessary given the problem statement only asks for the length. The line `return len(lis[::-1])` can be simplified to `return max(dp)`, as the `max(dp)` already holds the length of the LIS. This simplification removes redundant operations, improving the code's efficiency.

2. **Optimization of Dynamic Programming (DP) Table**: The DP table `dp` is initialized with all elements set to 1, assuming each single element is a subsequence of length 1. This initialization is correct, but the subsequent loops can be optimized. The current implementation has a runtime complexit

In [30]:
optimizer.step()

In [15]:
# Hopefully, we should get much better runtime!
longest_increasing_subsequence = run_function_in_interpreter(code.value)

start_time = time.time()
lis = longest_increasing_subsequence(nums)
end_time = time.time()

print(f"Longest Increasing Subsequence Length: {lis}")
print(f"Runtime: {end_time - start_time:.5f} seconds")

test_longest_increasing_subsequence(longest_increasing_subsequence)

Longest Increasing Subsequence Length: 180
Runtime: 2.39640 seconds
All test cases passed!


In [33]:
optimizer.zero_grad()
loss = loss_fn(problem, code)
loss.backward()
optimizer.step()

In [34]:
longest_increasing_subsequence = run_function_in_interpreter(code.value)

start_time = time.time()
lis = longest_increasing_subsequence(nums)
end_time = time.time()

print(f"Longest Increasing Subsequence Length: {lis}")
print(f"Runtime: {end_time - start_time:.5f} seconds")

test_longest_increasing_subsequence(longest_increasing_subsequence)

6
6
Longest Increasing Subsequence Length: 180
Runtime: 2.48514 seconds
All test cases passed!


In [35]:
print(code.value)

def longest_increasing_subsequence(nums: list[int]) -> int:
    """
    This function calculates the length of the longest increasing subsequence in a given list of integers.
    
    Args:
    nums (list[int]): A list of integers.
    
    Returns:
    int: The length of the longest increasing subsequence.
    """
    
    # Input validation
    if not isinstance(nums, list) or not all(isinstance(num, int) for num in nums):
        raise ValueError("Input must be a list of integers.")
    
    n = len(nums)
    if n == 0:
        return 0
    
    # Initialize the DP table
    dp = [1] * n
    
    # Compute the lengths of the longest increasing subsequences
    for i in range(1, n):
        for j in range(i):
            if nums[i] > nums[j]:
                dp[i] = max(dp[i], dp[j] + 1)
    
    # Return the length of the longest increasing subsequence
    return max(dp)

def longest_increasing_subsequence_binary_search(nums: list[int]) -> int:
    """
    This function calculates t